Disclaimer: Nothing herein is financial advice, and NOT a recommendation to trade real money. Many platforms exist for simulated trading (paper trading) which can be used for building and developing the methods discussed. Please use common sense and always first consult a professional before trading or investing.

<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/3-Practical/FinRL_PaperTrading_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a >

# Part 1: Install FinRL

In [50]:
## install finrl library
#%pip install --upgrade git+https://github.com/AI4Finance-Foundation/FinRL.git
%pip install --upgrade git+https://github.com/marcipops/FinRL.git@Refactor-FinRL_papertrading_demo

  Cloning https://github.com/marcipops/FinRL.git (to revision Refactor-FinRL_papertrading_demo) to /tmp/pip-req-build-9bwe1mk0
  Running command git clone -q https://github.com/marcipops/FinRL.git /tmp/pip-req-build-9bwe1mk0
286 minutes til market open.
  Running command git checkout -b Refactor-FinRL_papertrading_demo --track origin/Refactor-FinRL_papertrading_demo
  Switched to a new branch 'Refactor-FinRL_papertrading_demo'
  Branch 'Refactor-FinRL_papertrading_demo' set up to track remote branch 'Refactor-FinRL_papertrading_demo' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for finrl: filename=finrl-0.3.5-py3-none-any.whl size=2749646 sha256=2e36198e49c4cb8f707a5ed24c3e9241c4a076597daafdb82634e2bce3e2485d
  Stored in directory: /tmp/pip-ephem-wheel-cache-3kbwfb_x/wheels/d3/bb/ba/012aecc9467ccdca89cc387584e9510809aa067725368b0cb8
Successfully built finrl
  Attempting unins

## Get the API Keys Ready

In [51]:
API_KEY = "PKFZO57AZCL2J9TBL7LY"
API_SECRET = "LGXALuN2TgJX3dxXRsOLZXZgje16WH9of8cJfFGA"
API_BASE_URL = 'https://paper-api.alpaca.markets'
data_url = 'wss://data.alpaca.markets'

## Import related modules

In [52]:
from finrl.config_tickers import DOW_30_TICKER
from finrl.config import INDICATORS
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.meta.env_stock_trading.alpaca_papertrading import AlpacaPaperTrading
from finrl.meta.env_stock_trading.papertrading_common import train, test, alpaca_history, DIA_history

## Import Dow Jones 30 Symbols

In [53]:
ticker_list = DOW_30_TICKER
action_dim = len(DOW_30_TICKER)

In [54]:
print(ticker_list)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [55]:
print(INDICATORS)

['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']


## Calculate the DRL state dimension manually for paper trading

In [56]:
# amount + (turbulence, turbulence_bool) + (price, shares, cd (holding time)) * stock_dim + tech_dim
state_dim = 1 + 2 + 3 * action_dim + len(INDICATORS) * action_dim

In [57]:
state_dim

333

In [58]:
env = StockTradingEnv

# Part 2: Train the agent

## Train

In [59]:
ERL_PARAMS = {"learning_rate": 3e-6,"batch_size": 2048,"gamma":  0.985,
        "seed":312,"net_dimension":[128,64], "target_step":5000, "eval_gap":30,
        "eval_times":1} 
env = StockTradingEnv
#if you want to use larger datasets (change to longer period), and it raises error, 
#please try to increase "target_step". It should be larger than the episode steps. 

In [61]:
train(start_date = '2022-08-25', 
      end_date = '2022-08-31',
      ticker_list = ticker_list, 
      data_source = 'alpaca',
      time_interval= '1Min', 
      technical_indicator_list= INDICATORS,
      drl_lib='elegantrl', 
      env=env,
      model_name='ppo',
      if_vix=True, 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl', #current_working_dir
      break_step=1e5)

Alpaca successfully connected


KeyboardInterrupt: 

285 minutes til market open.
285 minutes til market open.
284 minutes til market open.


## Test

In [ ]:
account_value_erl=test(start_date = '2022-09-01', 
                      end_date = '2022-09-02',
                      ticker_list = ticker_list, 
                      data_source = 'alpaca',
                      time_interval= '1Min', 
                      technical_indicator_list= INDICATORS,
                      drl_lib='elegantrl', 
                      env=env, 
                      model_name='ppo',
                      if_vix=True, 
                      API_KEY = API_KEY, 
                      API_SECRET = API_SECRET, 
                      API_BASE_URL = API_BASE_URL,
                      cwd='./papertrading_erl',
                      net_dimension = ERL_PARAMS['net_dimension'])

Alpaca successfully connected
price_array:  780
| load actor from: ./papertrading_erl/actor.pth


/home/marc/venv/lib/python3.8/site-packages/finrl/meta/env_stock_trading/papertrading_common.py:522: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  s_tensor = _torch.as_tensor((state,), device=device)


Test Finished!
episode_return 1.0016788165466175


## Use full data to train 

After tuning well, retrain on the training and testing sets

In [ ]:
train(start_date = '2022-08-25', 
      end_date = '2022-09-02',
      ticker_list = ticker_list, 
      data_source = 'alpaca',
      time_interval= '1Min', 
      technical_indicator_list= INDICATORS,
      drl_lib='elegantrl', 
      env=env, 
      model_name='ppo',
      if_vix=True, 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl_retrain',
      break_step=2e5)

Alpaca successfully connected
939 minutes til market open.
938 minutes til market open.
937 minutes til market open.

| `step`: Number of samples, or total training steps, or running times of `env.step()`.
| `time`: Time spent from the start of training to this moment.
| `avgR`: Average value of cumulative rewards, which is the sum of rewards in an episode.
| `stdR`: Standard dev of cumulative rewards, which is the sum of rewards in an episode.
| `avgS`: Average of steps in an episode.
| `objC`: Objective of Critic network. Or call it loss function of critic network.
| `objA`: Objective of Actor network. It is the average Q value of the critic network.
|     step      time  |     avgR    stdR    avgS  |     objC      objA
| 2.00e+04        23  |   -28.10    0.00    2729  |     0.59      0.42
936 minutes til market open.
| 4.00e+04        47  |   -30.59    0.00    2729  |     0.46      0.42
| 6.00e+04        69  |   -30.94    0.00    2729  |     0.49      0.43
| 8.00e+04        92  |   

# Part 3: Deploy the agent

## Run Paper trading

In [ ]:
print(DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [ ]:
state_dim

333

In [ ]:
action_dim

30

In [ ]:
paper_trading_erl = AlpacaPaperTrading(ticker_list = DOW_30_TICKER, 
                                       time_interval = '1Min', 
                                       drl_lib = 'elegantrl', 
                                       agent = 'ppo', 
                                       cwd = './papertrading_erl_retrain', 
                                       net_dim = ERL_PARAMS['net_dimension'], 
                                       state_dim = state_dim, 
                                       action_dim= action_dim, 
                                       API_KEY = API_KEY, 
                                       API_SECRET = API_SECRET, 
                                       API_BASE_URL = API_BASE_URL, 
                                       tech_indicator_list = INDICATORS, 
                                       turbulence_thresh=30, 
                                       max_stock=1e2)
paper_trading_erl.run()

| load actor from: ./papertrading_erl_retrain/actor.pth
Waiting for market to open...
933 minutes til market open.
932 minutes til market open.
932 minutes til market open.
931 minutes til market open.
931 minutes til market open.
930 minutes til market open.
930 minutes til market open.
929 minutes til market open.
929 minutes til market open.
928 minutes til market open.
928 minutes til market open.
927 minutes til market open.
927 minutes til market open.
926 minutes til market open.
926 minutes til market open.
925 minutes til market open.
925 minutes til market open.
924 minutes til market open.
924 minutes til market open.
923 minutes til market open.
923 minutes til market open.
922 minutes til market open.
922 minutes til market open.
921 minutes til market open.
921 minutes til market open.
920 minutes til market open.
920 minutes til market open.
919 minutes til market open.
919 minutes til market open.
918 minutes til market open.
918 minutes til market open.
917 minutes til

KeyboardInterrupt: 

308 minutes til market open.
308 minutes til market open.
307 minutes til market open.
307 minutes til market open.
306 minutes til market open.
306 minutes til market open.
305 minutes til market open.
305 minutes til market open.
304 minutes til market open.
304 minutes til market open.
303 minutes til market open.
303 minutes til market open.
302 minutes til market open.
302 minutes til market open.
301 minutes til market open.
301 minutes til market open.
300 minutes til market open.
300 minutes til market open.
299 minutes til market open.
299 minutes til market open.
298 minutes til market open.
298 minutes til market open.
297 minutes til market open.
297 minutes til market open.
296 minutes til market open.
296 minutes til market open.
295 minutes til market open.
295 minutes til market open.
294 minutes til market open.
294 minutes til market open.
293 minutes til market open.
293 minutes til market open.
292 minutes til market open.
292 minutes til market open.
291 minutes ti

# Part 4: Check Portfolio Performance

## Get cumulative return

In [ ]:
df_erl, cumu_erl = alpaca_history(key=API_KEY, 
                                  secret=API_SECRET, 
                                  url=API_BASE_URL, 
                                  start='2022-09-01', #must be within 1 month
                                  end='2022-09-12') #change the date if error occurs


APIError: invalid timeframe provided: 5Min. Valid timeframe for days > 30 is 1D

In [ ]:
df_djia, cumu_djia = DIA_history(start='2022-09-01')

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ^DJI: 5m data not available for startTime=1662004800 and endTime=1670796611. The requested range must be within the last 60 days.


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
df_erl.tail()

NameError: name 'df_erl' is not defined

In [ ]:
returns_erl = cumu_erl -1 
returns_dia = cumu_djia - 1
returns_dia = returns_dia[:returns_erl.shape[0]]
print('len of erl return: ', returns_erl.shape[0])
print('len of dia return: ', returns_dia.shape[0])

NameError: name 'cumu_erl' is not defined

In [ ]:
returns_erl

## plot and save

In [ ]:
import matplotlib.pyplot as plt
plt.figure(dpi=1000)
plt.grid()
plt.grid(which='minor', axis='y')
plt.title('Stock Trading (Paper trading)', fontsize=20)
plt.plot(returns_erl, label = 'ElegantRL Agent', color = 'red')
#plt.plot(returns_sb3, label = 'Stable-Baselines3 Agent', color = 'blue' )
#plt.plot(returns_rllib, label = 'RLlib Agent', color = 'green')
plt.plot(returns_dia, label = 'DJIA', color = 'grey')
plt.ylabel('Return', fontsize=16)
plt.xlabel('Year 2021', fontsize=16)
plt.xticks(size = 14)
plt.yticks(size = 14)
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(78))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(6))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.005))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=2))
ax.xaxis.set_major_formatter(ticker.FixedFormatter(['','10-19','','10-20',
                                                    '','10-21','','10-22']))
plt.legend(fontsize=10.5)
plt.savefig('papertrading_stock.png')

NameError: name 'returns_erl' is not defined

979 minutes til market open.
978 minutes til market open.
977 minutes til market open.
976 minutes til market open.
975 minutes til market open.
974 minutes til market open.
973 minutes til market open.
972 minutes til market open.
971 minutes til market open.
970 minutes til market open.
969 minutes til market open.
968 minutes til market open.
967 minutes til market open.
966 minutes til market open.
965 minutes til market open.
964 minutes til market open.
963 minutes til market open.
962 minutes til market open.
961 minutes til market open.
960 minutes til market open.
959 minutes til market open.
958 minutes til market open.
957 minutes til market open.
956 minutes til market open.
954 minutes til market open.
953 minutes til market open.
952 minutes til market open.
951 minutes til market open.
950 minutes til market open.
949 minutes til market open.
948 minutes til market open.
947 minutes til market open.
946 minutes til market open.
945 minutes til market open.
